# Parameter Sweep
Last Edit: 07.19.2020 CYZ 

This code perfoms a parameter sweep

In [1]:
import numpy as np
import pandas as pd
from scipy import integrate

from utils.seir_model import seir_model_shields_rcfc_nolatent_time

# Define MCMC Wrapper

In [33]:
def seir_solve_parms(Thetas, CombPars):
    # Model
    t0 = 0
    tf = pars['nDays']
    tstep = 1 # day
    t = np.arange(t0,tf+tstep, tstep)
    
    CombPars[1]['R0'] = Thetas[0]
    CombPars[1]['q'] = Thetas[1]
    
    # Run Model
    model_out = integrate.odeint(seir_model_shields_rcfc_nolatent_time
                                 , X0
                                 , t
                                 , args=(CombPars,))#, rtol = 1, hmax=0.1, )
    
    Y = model_out[:,pars['D_ids']].sum(axis=1) 

    return(Y)

# Parameters

In [34]:
# Obtain defaults by executing 'set_default_parms.py' script
exec(open('utils/set_default_parms.py').read())

# returns: pars, epiPars, contactPars, intvPars, inits, X0

# Run Baseline Model

In [35]:
# Model
t0 = 0
tf = pars['nDays']
tstep = 1 # day
t = np.arange(t0,tf+tstep, tstep)
combPars = list((pars, epiPars, contactPars, intvPars))


# Run Model
model_out = integrate.odeint(seir_model_shields_rcfc_nolatent_time
                             , X0
                             , t
                             , args=(combPars,))#, rtol = 1, hmax=0.1, )

In [37]:
# corrupt model with noise
Y = model_out[:,pars['D_ids']].sum(axis=1) 
Y = np.round(Y/100)*100  # Rounding to hundredths might be enough noise.

# Let's fit R0 and q
thetas = np.zeros(2)
thetas
Y_prop = seir_solve_parms(thetas, combPars)

# Get Outputs

In [5]:
tot_deaths = np.sum(model_out[-1, pars['D_ids']])

tot_infec = np.sum(model_out[-1,pars['Isym_ids']]
                 + model_out[-1,pars['Iasym_ids']]
                 + model_out[-1,pars['Hsub_ids']]
                 + model_out[-1,pars['Hcri_ids']]
                 + model_out[-1,pars['D_ids']]
                 + model_out[-1,pars['R_ids']]
                 + model_out[-1,pars['Isym_pos_ids']]
                 + model_out[-1,pars['Iasym_pos_ids']]
                 + model_out[-1,pars['R_pos_ids']])

tot_cases = np.sum(model_out[-1,pars['Hsub_ids']]
                 + model_out[-1,pars['Hcri_ids']]
                 + model_out[-1,pars['D_ids']]
                 + model_out[-1,pars['Isym_pos_ids']]
                 + model_out[-1,pars['Iasym_pos_ids']]
                 + model_out[-1,pars['R_pos_ids']])

tot_released = np.sum(model_out[-1,pars['S_pos_ids']]
                  + model_out[-1,pars['E_pos_ids']]
                  + model_out[-1,pars['Isym_pos_ids']]
                  + model_out[-1,pars['Iasym_pos_ids']]
                  + model_out[-1,pars['R_pos_ids']])

In [6]:
tot_deaths

948634.678032529